### **Flights Data Exploration**

### 1. Connection

To connect to the Spark cluster, create a SparkSession object with the following params:

+ **appName:** FlightsDataExploration - The name of your Spark application, displayed in the Spark UI (e.g., `http://localhost:4040`).  Helps identify your application.

+ **spark.driver.memory:** 1g - `Memory` allocated to the `Spark driver` process, which coordinates application execution. 1 gigabytes.

+ **spark.executor.memory:** 1g - `Memory` allocated to each `Spark executor` process, where data processing and computations occur. 1 gigabytes.

+ **spark.sql.shuffle.partitions:** 200 - Number of `partitions` created during `shuffle operations` (e.g., joins, aggregations). 200 partitions.

+ **spark.sql.adaptive.enabled:** true - Enables `Adaptive Query Execution` (AQE) for dynamic query plan optimization. Enabled.

+ **spark.sql.autoBroadcastJoinThreshold:** 100mb - `Size threshold` for a table to be considered for `broadcast join`. 100 megabytes.

+ **spark.eventLog.enabled:** false - `Event Logging` on a spark history server.

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [4]:
# Initialize Spark Session with configurations
spark = (
    SparkSession.builder.appName("FlightsDataExplorationControlShufflePartitions")
    .config("spark.driver.memory", "1g")  # Driver memory
    .config("spark.executor.memory", "1g")  # Executor memory
    .config("spark.sql.shuffle.partitions", "70") # Control shuffle partitions
    .config("spark.sql.adaptive.enabled", "true")  # Enable adaptive query execution
    .config("spark.sql.autoBroadcastJoinThreshold", "50mb") # Adjust broadcast join threshold
    .config("spark.sql.files.maxPartitionBytes", "40mb") # Adjust max size per partition to control partition count
    .config("spark.eventLog.enabled", "false") # for some debug app will be disabled
    .getOrCreate()
)

### 2. Read

This section focuses on reading the various datasets required for our flight analysis. We utilize the `spark.read.csv()` function to load data from CSV files into Spark DataFrames.  The `header=True` option specifies that the first row of each CSV file contains column names.  `inferSchema=True` tells Spark to attempt to automatically determine the data type of each column (e.g., string, integer, double).

Specifically, we load the following datasets:

* **Airlines Dataset:**  `airlines_df` contains information about airlines, including their IATA codes and full names.  It is read from the `airlines.csv` file.

* **Airports Dataset:** `airports_df` contains details about airports, such as their IATA codes, names, city, state, and location.  It is read from the `airports.csv` file.

* **Cancellation Codes Dataset:** `cancellation_codes_df` provides descriptions for different cancellation codes. It is read from the `cancellation_codes.csv` file.

* **Flights Dataset:** `flights_df` contains detailed information about individual flights, including dates, times, airlines, origin and destination airports, delays, and cancellation status. It is read from the `flights.csv` file.

These DataFrames will be used in subsequent sections for data exploration, transformations, and analysis.

In [5]:
# --- Read Data ---
airlines_df = spark.read \
  .csv(
    "data/flights/airlines.csv",
    header=True,
    inferSchema=True
  )
airports_df = spark.read \
  .csv(
    "data/flights/airports.csv",
    header = True,
    inferSchema = True
  )
cancellation_codes_df = spark.read \
  .csv(
    "data/flights/cancellation_codes.csv",
    header = True,
    inferSchema = True
  )

**Reading Flight Data with Schema Inference**

This code demonstrates an efficient way to read flight data from a CSV file using Spark, leveraging schema inference for optimal performance.


In [6]:
sampled_flights_df = spark.read \
  .csv(
    "data/flights/flights.csv",
    header = True,
    inferSchema = True
  ) \
  .sample(
    withReplacement = False,
    fraction = 0.000001,
    seed = 42
  )

flights_schema = sampled_flights_df.schema

flights_df = spark.read \
  .csv(
    "data/flights/flights.csv",
    header = True,
    schema=flights_schema
  )

### 3. Data Exploration and Transformations

This section demonstrates various data exploration and transformation techniques using PySpark to analyze the flight data. We perform `aggregations`, `filtering`, `joins`, and other operations to gain insights into the data.  Because we are using a sample of the data, the results shown here will reflect the properties of the sample, not the full dataset.  If you wish to run these analyses on the full dataset, remove the `.sample()` operation and the `caching/persisting`.

### 1. Which airline had the most cancellations?

* **Finding the Airline with Most Cancellations:** We filter the `flights_df` to isolate cancelled flights (`CANCELLED` == 1). Then, we group the cancelled flights by `AIRLINE` and count the number of cancellations for each airline. The results are ordered in descending order of cancellation count to identify the airline with the most cancellations.

* **Joining with Airline Names:** To provide more context, we join the cancellation counts with the `airlines_df` using the `AIRLINE` IATA code. This adds the full airline name to the cancellation counts, making the results easier to interpret.


In [7]:
cancelled_flights = flights_df.filter(
  col("CANCELLED") == 1
)

cancellation_counts = (
    cancelled_flights.groupBy("AIRLINE")
    .agg(
      count("*").alias("cancellation_count")
    )
    .orderBy(
      col("cancellation_count").desc()
    )
)

cancellation_counts.show()

+-------+------------------+
|AIRLINE|cancellation_count|
+-------+------------------+
|     WN|             16043|
|     EV|             15231|
|     MQ|             15025|
|     AA|             10919|
|     OO|              9960|
|     UA|              6573|
|     B6|              4276|
|     US|              4067|
|     DL|              3824|
|     NK|              2004|
|     AS|               669|
|     F9|               588|
|     VX|               534|
|     HA|               171|
+-------+------------------+



In [ ]:
# Join with airline names for better readability
cancellation_counts_with_names = cancellation_counts.join(
    airlines_df,
    cancellation_counts["AIRLINE"] == airlines_df["IATA_CODE"],
    "inner"
).select(airlines_df["AIRLINE"], "cancellation_count")

cancellation_counts_with_names.show()

+--------------------+------------------+
|             AIRLINE|cancellation_count|
+--------------------+------------------+
|Southwest Airline...|             16043|
|Delta Air Lines Inc.|              3824|
|     JetBlue Airways|              4276|
|American Airlines...|             10919|
|American Eagle Ai...|             15025|
|Skywest Airlines ...|              9960|
|      Virgin America|               534|
|United Air Lines ...|              6573|
|Atlantic Southeas...|             15231|
|    Spirit Air Lines|              2004|
|Hawaiian Airlines...|               171|
|Frontier Airlines...|               588|
|     US Airways Inc.|              4067|
|Alaska Airlines Inc.|               669|
+--------------------+------------------+



In [27]:
cancellation_counts_with_names.explain("formatted")

== Physical Plan ==
AdaptiveSparkPlan (12)
+- Project (11)
   +- BroadcastHashJoin Inner BuildRight (10)
      :- HashAggregate (6)
      :  +- Exchange (5)
      :     +- HashAggregate (4)
      :        +- Project (3)
      :           +- Filter (2)
      :              +- Scan csv  (1)
      +- BroadcastExchange (9)
         +- Filter (8)
            +- Scan csv  (7)


(1) Scan csv 
Output [2]: [AIRLINE#798, CANCELLED#818]
Batched: false
Location: InMemoryFileIndex [file:/opt/workspace/data/flights/flights.csv]
PushedFilters: [IsNotNull(CANCELLED), EqualTo(CANCELLED,1), IsNotNull(AIRLINE)]
ReadSchema: struct<AIRLINE:string,CANCELLED:int>

(2) Filter
Input [2]: [AIRLINE#798, CANCELLED#818]
Condition : ((isnotnull(CANCELLED#818) AND (CANCELLED#818 = 1)) AND isnotnull(AIRLINE#798))

(3) Project
Output [1]: [AIRLINE#798]
Input [2]: [AIRLINE#798, CANCELLED#818]

(4) HashAggregate
Input [1]: [AIRLINE#798]
Keys [1]: [AIRLINE#798]
Functions [1]: [partial_count(1)]
Aggregate Attributes [1]: 

25/02/26 12:48:16 ERROR TaskSchedulerImpl: Lost executor 2 on 192.168.144.6: worker lost: Not receiving heartbeat for 60 seconds
25/02/26 12:48:16 ERROR TaskSchedulerImpl: Lost executor 1 on 192.168.144.5: worker lost: Not receiving heartbeat for 60 seconds
25/02/26 12:48:16 ERROR TaskSchedulerImpl: Lost executor 0 on 192.168.144.4: worker lost: Not receiving heartbeat for 60 seconds


### 2. What are top 10 Airlines with most flights:**

* **Counting Flights per Airline:** We group the `flights_df` by `AIRLINE` and count the number of flights for each airline.

* **Limiting to Top 10:** We use the `limit()` function to select only the top 10 airlines with the most flights.

* **Ordering and Displaying Results:** The results are ordered by flight count in descending order.


In [24]:
top_airlines = (
    flights_df.groupBy("AIRLINE")
    .agg(count("*").alias("flight_count"))
    .orderBy(col("flight_count").desc())
    .limit(10)  # Limit to top 10
)
top_airlines.show()

+-------+------------+
|AIRLINE|flight_count|
+-------+------------+
|     WN|     1261855|
|     DL|      875881|
|     AA|      725984|
|     OO|      588353|
|     EV|      571977|
|     UA|      515723|
|     MQ|      294632|
|     B6|      267048|
|     US|      198715|
|     AS|      172521|
+-------+------------+



### 3. What were the top 10 busiest airports (most flights)?

* **Calculating Airport Traffic:** We determine the busiest airports by calculating the total number of flights (arrivals and departures) for each airport. We group the `flights_df` by `ORIGIN_AIRPORT` and `DESTINATION_AIRPORT` separately to count departures and arrivals.

* **Combining Arrival and Departure Counts:** We perform a full outer join on the origin and destination counts to get the combined traffic for each airport.  The `coalesce` function is used to handle cases where an airport might only have arrivals or departures, ensuring that all airports are included in the results. The total traffic is calculated by summing the origin and destination counts.

* **Ordering and Displaying Results:** The results are ordered by total traffic in descending order to show the busiest airports.


In [6]:
origin_counts = flights_df.groupBy("ORIGIN_AIRPORT") \
  .agg(
    count("*")
    .alias("origin_count")
  )
destination_counts = flights_df.groupBy("DESTINATION_AIRPORT") \
  .agg(
    count("*")
    .alias("destination_count")
  )


In [ ]:
airport_traffic = origin_counts.join(
  destination_counts,
  origin_counts["ORIGIN_AIRPORT"] == destination_counts["DESTINATION_AIRPORT"],
  "fullouter"
  ) \
  .select(
    coalesce(
      col("ORIGIN_AIRPORT"),
      col("DESTINATION_AIRPORT")
    ).alias("airport"),
    coalesce(
      origin_counts["origin_count"],
      lit(0)
    ).alias("origin_count"),
    coalesce(
      destination_counts["destination_count"],
      lit(0)
    ).alias("destination_count")) \
  .withColumn(
    "total_traffic",
    col("origin_count") + col("destination_count")
  ) \
  .orderBy(
    col("total_traffic").desc()
  ).explain(True)

# airport_traffic.show(10)

== Parsed Logical Plan ==
'Sort ['total_traffic DESC NULLS LAST], true
+- Project [airport#325, origin_count#326L, destination_count#327L, (origin_count#326L + destination_count#327L) AS total_traffic#331L]
   +- Project [coalesce(ORIGIN_AIRPORT#159, DESTINATION_AIRPORT#293) AS airport#325, coalesce(origin_count#246L, cast(0 as bigint)) AS origin_count#326L, coalesce(destination_count#282L, cast(0 as bigint)) AS destination_count#327L]
      +- Join FullOuter, (ORIGIN_AIRPORT#159 = DESTINATION_AIRPORT#293)
         :- Aggregate [ORIGIN_AIRPORT#159], [ORIGIN_AIRPORT#159, count(1) AS origin_count#246L]
         :  +- Relation [YEAR#152,MONTH#153,DAY#154,DAY_OF_WEEK#155,AIRLINE#156,FLIGHT_NUMBER#157,TAIL_NUMBER#158,ORIGIN_AIRPORT#159,DESTINATION_AIRPORT#160,SCHEDULED_DEPARTURE#161,DEPARTURE_TIME#162,DEPARTURE_DELAY#163,TAXI_OUT#164,WHEELS_OFF#165,SCHEDULED_TIME#166,ELAPSED_TIME#167,AIR_TIME#168,DISTANCE#169,WHEELS_ON#170,TAXI_IN#171,SCHEDULED_ARRIVAL#172,ARRIVAL_TIME#173,ARRIVAL_DELAY#174

25/02/21 12:22:15 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


### 4. Stop the application

In [15]:
spark.stop()